### Load experiment metadata

In [11]:
from sumo_python.helpers.load_experiment_metadata import load_experiment_metadata

input_path = "../network/quickstart/metadata/"


In [12]:
import os
os.listdir(input_path)

['config.json', 'simulation_setups.json']

In [13]:
config, sim_setup = load_experiment_metadata(input_path)

In [14]:

config

{'NETWORK': PosixPath('../network/quickstart'),
 'SUMO': PosixPath('/usr/local/opt/sumo/share/sumo')}

In [15]:
sim_setup

{'objective': 'counts',
 'net': 'quickstart.net.xml',
 'route2od': 'quickstart.rou_for_od.xml',
 'taz': 'quickstart.taz.xml',
 'add': 'quickstart.additional.xml',
 'start_sim_sec': 54000,
 'end_sim_sec': 57600,
 'gt_od': 'quickstart.gt_od.xml',
 'current_od': 'quickstart.current_od.xml',
 'init_od': 'quickstart.init_od.xml',
 'prior_od': 'quickstart.prior_od.xml',
 'n_sumo_replicate': 1}

# Simulator

Steps:

1. Create OD meta-files that for a given count, creates a random trajectoy of vehicles moving from O -> D
2. Run SUMO against the latter files
3. Parse output of simulation and yields the aggregated value of vehicles which arrive to a given node in th network



### Step 1. Create OD metadata

In [16]:
import numpy as np
from sumo_python.helpers.generate_od_xml import generate_od_xml


prefix_run = 'iter'
#od_rand = np.random.rand(4)*200
od_rand = np.array([500,500,900,900])
#print(prefix_run, np.sum(od_rand))
    
generate_od_xml(od_rand, config, sim_setup, prefix_run)


Reading: ../network/quickstart/quickstart.init_od.xml
Saving: ../network/quickstart/iter_quickstart.current_od.xml


### Step 2-3: Run SUMO and parse output

In [17]:
from sumo_python.simulator.run_sumo_and_parse_output import run_sumo_and_parse_output

df_simulated = run_sumo_and_parse_output(config, sim_setup, prefix_run)


### Runing: iter
Success.
Step #57600.00 (8ms ~= 125.00*RT, ~82000.00UPS, vehicles TOT 2800 ACT 656 BUF 0)          


In [18]:
df_simulated

,EdgeID,interval_begin,interval_end,interval_nVehContrib,interval_harmonicMeanSpeed
0,#EdgeL5,54000.0,57600.0,959.0,13.500000
1,#EdgeL7,54000.0,57600.0,1243.0,13.603333


### Obtain ground-truth

In [19]:
from sumo_python.simulator.ground_truth import run_ground_truth_and_parse
df_gt_data = run_ground_truth_and_parse(config, sim_setup)

od2trips --no-step-log  --spread.uniform --taz-files ../network/quickstart/quickstart.taz.xml --tazrelation-files ../network/quickstart/quickstart.gt_od.xml -o ../network/quickstart/gt_od_trips.trips.xml 
Success.
sumo --output-prefix gt_ --ignore-route-errors=true --net-file=../network/quickstart/quickstart.net.xml --routes=../network/quickstart/gt_od_trips.trips.xml -b 54000 -e 57600 --additional-files ../network/quickstart/quickstart.additional.xml --duration-log.statistics --xml-validation never --vehroutes ../network/quickstart/gt_routes.vehroutes.xml 
Loading net-file from '../network/quickstart/quickstart.net.xml' ... done (9ms).
Loading additional-files from '../network/quickstart/quickstart.additional.xml' ... done (2ms).
Loading done.
Simulation version 1.16.0 started with time: 54000.00.
Step #57600.00 (7ms ~= 142.86*RT, ~93714.29UPS, vehicles TOT 2800 ACT 656 BUF 0)          
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance:

In [20]:
df_gt_data

,EdgeID,interval_begin,interval_end,interval_nVehContrib,interval_harmonicMeanSpeed
0,#EdgeL5,54000.0,57600.0,959,13.500000
1,#EdgeL7,54000.0,57600.0,1243,13.603333


## Compute loss

In [21]:
from sumo_python.helpers.nrmse_counts import compute_nrmse_counts

loss_estimate = compute_nrmse_counts(df_gt_data, df_simulated)
loss_estimate

0.0